In [ ]:
import gymnasium as gym
import numpy as np
from gymnasium import spaces

from lib import sim_env as sim
from lib import loads
from lib.utils import (DTParser, current_time_tz, 
                       dt_to_timezone, adjust_db_dates_local_tz)
from lib import agents
import datetime

In [ ]:
def build_sim_env() -> tuple[
    sim.SimulationEnvironment,
    sim.InfrastructureManager,
    sim.Dispatcher,
    agents.AllocationAgent,
]:
    # !! REWORK, CURRENTLY ONLY FOR TESTING PURPOSES
    """Constructor to build simulation environment (layout)

    Returns
    -------
    tuple[ sim.SimulationEnvironment, sim.InfrastructureManager, sim.Dispatcher, agents.AllocationAgent, ]
        tuple out of Environment, InfrastructureManager, Dispatcher, Agent
    """
    starting_dt = current_time_tz(cut_microseconds=True)
    
    env = sim.SimulationEnvironment(name='base', time_unit='seconds', starting_datetime=dt)
    job_generator = loads.RandomJobGenerator(seed=2)
    infstruct_mgr = sim.InfrastructureManager(env=env)
    dispatcher = sim.Dispatcher(env=env, priority_rule='FIFO')
    
    # source
    area_source = sim.ProductionArea(env=env, custom_identifier=1000)
    group_source = sim.StationGroup(env=env, custom_identifier=1000)
    area_source.add_subsystem(group_source)
    proc_time = dt_parser.timedelta_from_val(val=2., time_unit='hours')
    source = sim.Source(env=env, custom_identifier='source', proc_time=proc_time, 
                    random_generation=True, job_generator=job_generator, num_gen_jobs=5)
    group_source.add_subsystem(source)

    # sink
    area_sink = sim.ProductionArea(env=env, custom_identifier=2000)
    group_sink = sim.StationGroup(env=env, custom_identifier=2000)
    area_sink.add_subsystem(group_sink)
    sink = sim.Sink(env=env, custom_identifier='sink')
    group_sink.add_subsystem(sink)

    # processing stations
    # prod area 1
    area_prod = sim.ProductionArea(env=env, custom_identifier=1)
    group_prod = sim.StationGroup(env=env, custom_identifier=1)
    area_prod.add_subsystem(group_prod)
    group_prod2 = sim.StationGroup(env=env, custom_identifier=2)
    area_prod.add_subsystem(group_prod2)
    # prod area 2
    #area_prod2 = ProductionArea(env=env, custom_identifier=2)
    #group_prod3 = StationGroup(env=env, custom_identifier=3)
    #area_prod2.add_subsystem(group_prod3)
    #area_prod.add_subsystem(group_prod3)
    ## machines
    for machine in range(3):
        buffer = sim.Buffer(capacity=20, env=env, custom_identifier=(10+machine))
        if machine == 5:
            MachInst = sim.Machine(env=env, custom_identifier=machine, buffers=[buffer], setup_time=5.)
        else:
            MachInst = sim.Machine(env=env, custom_identifier=machine, buffers=[buffer])
            
        if machine == 0:
            testMachInst = MachInst
        
        if machine < 2:
            group_prod.add_subsystem(buffer)
            group_prod.add_subsystem(MachInst)
        elif machine >= 2:
            group_prod2.add_subsystem(buffer)
            group_prod2.add_subsystem(MachInst)
        else:
            pass
            #group_prod3.add_subsystem(buffer)
            #group_prod3.add_subsystem(MachInst)
        

    add_machine_to_bottleneck: bool = False
    if add_machine_to_bottleneck:
        buffer = sim.Buffer(capacity=20, env=env, custom_identifier=(10+machine+1))
        MachInst = sim.Machine(env=env, custom_identifier=machine+1, buffers=[buffer])
        group_prod3.add_subsystem(buffer)
        group_prod3.add_subsystem(MachInst)
        
    alloc_agent = agents.AllocationAgent(assoc_system=area_prod)
    
    return env, infstruct_mgr, dispatcher, alloc_agent

In [ ]:
class JSSEnv(gym.Env):
    """Custom Environment that follows gym interface."""

    metadata = {"render_modes": ["human"], "render_fps": 30}

    def __init__(self, arg1, arg2, ...):
        super().__init__()
        
        # build env
        (self.sim_env, self.infstruct_mgr, 
         self.dispatcher, self.agent) = build_sim_env()
        # action space for allocation agent is length of all associated 
        # infrastructure objects
        n_actions = len(self.agent.assoc_infstrct_objs)
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        # number of discrete actions depends on layout and infrastructure
        self.action_space = spaces.Discrete(n_actions)
        # Example for using image as input (channel-first; channel-last also works):
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(N_CHANNELS, HEIGHT, WIDTH), dtype=np.uint8)

    def step(self, action):
        ...
        # process given action
        # step through sim_env till new decision should be made
        # calculate reward based on new observation
        return observation, reward, terminated, truncated, info

    def reset(self, seed=None, options=None):
        
        # re-init simulation environment
        (self.sim_env, self.infstruct_mgr, 
         self.dispatcher, self.agent) = build_sim_env()
        
        # run till first decision should be made
        # use ramp-up phase, change allocation rule if condition is met
        
        return observation, info

    def render(self):
        ...

    def close(self):
        ...